In [1]:
import json
import os
from typing import Dict, Union

import pandas as pd
from tqdm.notebook import tqdm

CountDict = Dict[Union[int, str], int]

data_dir = 'data/avazu'
os.listdir(data_dir)

['train.bz2',
 'train_counts.json',
 'test',
 'processed.tar.gz',
 'train',
 'processed',
 'processed-csv',
 'size.json']

In [2]:
import math

n = 40428967
n_chunk = 10
chunksize = math.ceil(n / n_chunk)
reader = pd.read_csv(os.path.join(data_dir, 'train'), chunksize=chunksize)

In [3]:
from process.categorical import get_count_dict

counts = get_count_dict(os.path.join(data_dir, 'train_counts.json'))

In [4]:
trans_dicts = {c: {k: i + 1 for i, k in enumerate(d)} for c, d in tqdm(counts.items())}

In [5]:
size_dict = {c: max(d.values()) + 1 for c, d in trans_dicts.items()}

In [6]:
with open(os.path.join(data_dir, 'size.json'), 'w') as f:
    json.dump(size_dict, f)

In [7]:
import pickle

processed_dir = os.path.join(data_dir, 'processed')
os.makedirs(processed_dir, exist_ok=True)
i = 0
for df_i in tqdm(reader):
    for c, dic in tqdm(trans_dicts.items()):
        df_i[c] = df_i[c].map(lambda x: dic.get(x, 0))
    with open(os.path.join(processed_dir, 'dataframe-{}'.format(i)), 'wb') as f:
              pickle.dump(df_i, f)
    i += 1